In [2]:
import rasterio
import numpy as np
from pyproj import Proj, transform
from rasterio.transform import from_origin
import ephem
import datetime
import math

### Function to get the lat long from the drone image
### Load drone data path in the function

In [3]:
def get_coordinates(file_path):
    # Load geotiff
    with rasterio.open(file_path) as r:
        T0 = r.transform  # upper-left pixel corner affine transform
        p1 = Proj(r.crs)
        A = r.read()  # pixel values

    # All rows and columns
    cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

    # Get pixel coordinates from the rows and columns
    T1 = T0 * rasterio.Affine.scale(1, -1)
    rc2en = lambda r, c: (c, r) * T1  
    eastings, northings = np.vectorize(rc2en, otypes=[float, float])(rows, cols)

    p2 = Proj(proj='latlong', datum='WGS84')
    longs, lats = transform(p1, p2, eastings, northings)

    return longs, lats

file_path = r'C:\Users\JANUSHI SHASTRI\Desktop\CIRES_PROJECT\Drone Data\drive-download-20230711T205208Z-001\niwot_6_23_2022_ortho_cropped.tif'
longitudes, latitudes = get_coordinates(file_path)

print('Longitudes:', longitudes)
print('Latitudes:', latitudes)

C:\Users\JANUSHI SHASTRI\AppData\Local\Temp\ipykernel_48828\3174482380.py:13: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  rc2en = lambda r, c: (c, r) * T1
C:\Users\JANUSHI SHASTRI\AppData\Local\Temp\ipykernel_48828\3174482380.py:17: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longs, lats = transform(p1, p2, eastings, northings)


Longitudes: [[-105.55816094 -105.55816011 -105.55815927 ... -105.55651632
  -105.55651549 -105.55651466]
 [-105.55816094 -105.55816011 -105.55815928 ... -105.55651633
  -105.5565155  -105.55651466]
 [-105.55816095 -105.55816012 -105.55815928 ... -105.55651633
  -105.5565155  -105.55651467]
 ...
 [-105.55817339 -105.55817256 -105.55817173 ... -105.55652874
  -105.55652791 -105.55652707]
 [-105.5581734  -105.55817256 -105.55817173 ... -105.55652875
  -105.55652791 -105.55652708]
 [-105.5581734  -105.55817257 -105.55817174 ... -105.55652875
  -105.55652792 -105.55652709]]
Latitudes: [[40.03941242 40.03941242 40.03941243 ... 40.03942033 40.03942033
  40.03942034]
 [40.03941306 40.03941306 40.03941307 ... 40.03942097 40.03942097
  40.03942098]
 [40.0394137  40.0394137  40.03941371 ... 40.03942161 40.03942161
  40.03942162]
 ...
 [40.04093971 40.04093971 40.04093971 ... 40.04094762 40.04094762
  40.04094763]
 [40.04094035 40.04094035 40.04094035 ... 40.04094826 40.04094826
  40.04094827]
 [4

### Method 1:

In [8]:
# Calculate the central latitude and longitude
center_long = np.mean(longitudes)
center_lat = np.mean(latitudes)

# Convert the date and time to UTC. The time given is 2:34 PM, which is 14:34 in 24-hour format
date_time_str = '2022-06-23 14:34:00'

# Calculation of sun azimuth and zenith angle
# Ephemeris computations
observer = ephem.Observer()
observer.lat = str(center_lat)
observer.lon = str(center_long)
observer.date = ephem.Date(date_time_str)

sun = ephem.Sun()
sun.compute(observer)

azimuth = sun.az
zenith = ephem.degrees('90') - sun.alt
azimuth_deg = math.degrees(azimuth)
zenith_deg = math.degrees(zenith)

print(f'Sun azimuth in degrees: {azimuth_deg}, Sun zenith in degrees: {zenith_deg}')
# print(f'Sun azimuth: {azimuth}, Sun zenith: {zenith}')

Sun azimuth in degrees: 84.64035306276917, Sun zenith in degrees: 58.434557373447454


### Method 2

### Function to calculate sun angles using all pixels of image and lat long

In [4]:
def get_sun_angles(file_path, date_time_str):
    # Load geotiff
    with rasterio.open(file_path) as r:
        T0 = r.transform  # upper-left pixel corner affine transform
        p1 = Proj(r.crs)
        A = r.read()  # pixel values

    # All rows and columns
    cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

    # Get pixel coordinates from the rows and columns
    T1 = T0 * rasterio.Affine.scale(1, -1)
    rc2en = lambda r, c: (c, r) * T1  
    eastings, northings = np.vectorize(rc2en, otypes=[float, float])(rows, cols)

    p2 = Proj(proj='latlong',datum='WGS84')
    longs, lats = transform(p1, p2, eastings, northings)

    # Calculation of sun azimuth and zenith angle
    # We'll do this for the center of the image
    center_lat = np.mean(lats)
    center_long = np.mean(longs)

    # Convert the date and time to UTC.
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

    # Ephemeris computations
    observer = ephem.Observer()
    observer.lat = str(center_lat)
    observer.lon = str(center_long)
    observer.date = ephem.Date(date_time_obj)

    sun = ephem.Sun()
    sun.compute(observer)

    azimuth = sun.az
    zenith = ephem.degrees('90') - sun.alt
    # Convert radians to degrees
    azimuth_deg = math.degrees(azimuth)
    zenith_deg = math.degrees(zenith)

    print(f'Sun azimuth in degrees: {azimuth_deg}, Sun zenith in degrees: {zenith_deg}')

    return azimuth, zenith

file_path = r'C:\Users\JANUSHI SHASTRI\Desktop\CIRES_PROJECT\Drone Data\drive-download-20230711T205208Z-001\niwot_6_23_2022_ortho_cropped.tif'
date_time_str = '2022-06-23 14:34:00'

azimuth, zenith = get_sun_angles(file_path, date_time_str)

# print(f'Sun azimuth: {azimuth}, Sun zenith: {zenith}')

C:\Users\JANUSHI SHASTRI\AppData\Local\Temp\ipykernel_46172\550025637.py:13: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  rc2en = lambda r, c: (c, r) * T1
C:\Users\JANUSHI SHASTRI\AppData\Local\Temp\ipykernel_46172\550025637.py:17: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longs, lats = transform(p1, p2, eastings, northings)


Sun azimuth in degrees: 84.64035306276917, Sun zenith in degrees: 58.434557373447454
